# Train and Check accuracy on 20% test data

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/URL_Checker")  # this appends the parent directory to the sys.path list
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import json
import tensorflow as tf
from nltk.util import ngrams
import itertools
from Utils import Evaluation
from Utils import DataProcessing, FeatureCreation, Model
N_for_NGram = 3
Sequence_length = 50
epochs = 20
n_classes = 2

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
from sklearn.model_selection import train_test_split
df1 = pd.read_csv("/content/drive/MyDrive/URL_Checker/dataset/df_final.csv")
df2 = pd.read_csv("/content/drive/MyDrive/URL_Checker/dataset/all_top_domains_merged.csv")
test_df = pd.read_csv("/content/drive/MyDrive/URL_Checker/dataset/df_another_validation.csv")
url_dataframe = pd.concat([df1,df2], axis=0)
url_dataframe = url_dataframe.sample(frac=1).reset_index(drop=True)
url_dataframe = url_dataframe.drop_duplicates()
url_dataframe = url_dataframe.dropna().reset_index(drop=True)

In [ ]:
train_df, val_df = train_test_split(url_dataframe, test_size=0.20, stratify=url_dataframe['type'])
# train_df, val_df = train_test_split(train_df, test_size=0.20, stratify=train_df['type'])

In [ ]:
Batch_Size = 16 * strategy.num_replicas_in_sync
# Batch_Size = 128
max_tokens = 10000
with strategy.scope():
  train_ds = DataProcessing.process_train_data(train_df,Batch_Size)
  Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                    split="character",
                                                    ngrams=(N_for_NGram,),
                                                    output_mode='int',
                                                    max_tokens = max_tokens,
                                                    output_sequence_length=Sequence_length)
  # Make a text-only dataset (without labels), then call adapt
  train_text = train_ds.map(lambda x,z: x[0])
  Vectorize_Layer.adapt(train_text)
  train_ds = train_ds.map(lambda x, z: DataProcessing.vectorize_text(x[0],x[1],z,Vectorize_Layer))
  train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
with strategy.scope():
  val_ds = DataProcessing.process_data(val_df,Batch_Size,Vectorize_Layer)
  test_ds = DataProcessing.process_data(test_df,Batch_Size,Vectorize_Layer)

In [ ]:
with strategy.scope():
  model = Model.create_model(Sequence_length, max_tokens, 18)
  print(model.summary())
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 16)       160000      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 gru (GRU)                      (None, 16)           1632        ['embedding[0][0]']              
                                                                                            

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    batch_size=Batch_Size,
    validation_data=val_ds)

Epoch 1/20
4846/4846 [==============================] - 144s 28ms/step - loss: 0.1991 - accuracy: 0.9298 - val_loss: 0.0744 - val_accuracy: 0.9750
Epoch 2/20
4846/4846 [==============================] - 123s 25ms/step - loss: 0.0809 - accuracy: 0.9732 - val_loss: 0.0627 - val_accuracy: 0.9783
Epoch 3/20
4846/4846 [==============================] - 122s 25ms/step - loss: 0.0689 - accuracy: 0.9771 - val_loss: 0.0592 - val_accuracy: 0.9800
Epoch 4/20
4846/4846 [==============================] - 121s 25ms/step - loss: 0.0621 - accuracy: 0.9797 - val_loss: 0.0536 - val_accuracy: 0.9820
Epoch 5/20
4846/4846 [==============================] - 121s 25ms/step - loss: 0.0567 - accuracy: 0.9813 - val_loss: 0.0552 - val_accuracy: 0.9817
Epoch 6/20
4846/4846 [==============================] - 120s 25ms/step - loss: 0.0525 - accuracy: 0.9830 - val_loss: 0.0499 - val_accuracy: 0.9832
Epoch 7/20
4846/4846 [==============================] - 119s 25ms/step - loss: 0.0486 - accuracy: 0.9841 - val_loss: 0

In [ ]:
result = Evaluation.evaluate_model(model, test_ds, 0.5)

Exception ignored in: <function Executor.__del__ at 0x7ce86885b880>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.OutOfRangeError: End of sequence


3901/3901 [==============================] - 64s 16ms/step
Confusion Matrix:
[[  8520  98103]
 [ 46160 346534]]
Accuracy: 0.7110793343707504
Precision: 0.6462127998444198
Recall: 0.7110793343707504
F1 Score: 0.6735214855648405


Here it represents over 98.4% accuracy on validation data i.e. 20% of total data. and 71% accuracy for another dataset, which is not belongs to our dataset (it means our model is more generalized and will perform great for unseen data.)

In [ ]:
links = ["itunes.apple.com/gb/album/whb-bonus-track-version/id358709625", "http://45.231.210.144:47317/Mozi.m", "www.google.com"]

In [ ]:
Evaluation.make_prediction(model,Batch_Size, Vectorize_Layer, links, 0.5)

1/1 [==============================] - 2s 2s/step


['Safe', 'Malicious', 'Safe']

# Training the model on complete Dataset

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/URL_Checker/dataset/df_final.csv")
df2 = pd.read_csv("/content/drive/MyDrive/URL_Checker/dataset/all_top_domains_merged.csv")
df3 = pd.read_csv("/content/drive/MyDrive/URL_Checker/dataset/df_another_validation.csv")
url_dataframe = pd.concat([df1,df2, df3], axis=0)
url_dataframe = url_dataframe.sample(frac=1).reset_index(drop=True)
url_dataframe = url_dataframe.drop_duplicates()
url_dataframe = url_dataframe.dropna().reset_index(drop=True)
Batch_Size = 16 * strategy.num_replicas_in_sync
# Batch_Size = 128
N_for_NGram = 3
Sequence_length = 50
epochs = 20
n_classes = 2
max_tokens = 10000
with strategy.scope():
  train_ds = DataProcessing.process_train_data(url_dataframe,Batch_Size)
  Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                    split="character",
                                                    ngrams=(N_for_NGram,),
                                                    output_mode='int',
                                                    max_tokens = max_tokens,
                                                    output_sequence_length=Sequence_length)

  # Make a text-only dataset (without labels), then call adapt
  train_text = train_ds.map(lambda x,z: x[0])
  Vectorize_Layer.adapt(train_text)
  train_ds = train_ds.map(lambda x, z: DataProcessing.vectorize_text(x[0],x[1],z,Vectorize_Layer))
  train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

  model = Model.create_model(Sequence_length, max_tokens, 18)
  print(model.summary())
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    train_ds,
    epochs=epochs,
    batch_size=Batch_Size)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 16)       160000      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 gru (GRU)                      (None, 16)           1632        ['embedding[0][0]']              
                                                                                            

In [ ]:
links = ["itunes.apple.com/gb/album/whb-bonus-track-version/id358709625","http://45.231.210.144:47317/Mozi.m","youtu.be"]
Evaluation.make_prediction(model,1, Vectorize_Layer, links, 0.5)

1/1 [==============================] - 2s 2s/step


['Safe', 'Malicious', 'Safe']

# Save the Model and Vocabulary of Vectorization Layer

In [ ]:
model.save("/content/drive/MyDrive/URL_Checker/Trained_Model/Model/Model.keras")

In [ ]:
import json
vocab = Vectorize_Layer.get_vocabulary()
with open("/content/drive/MyDrive/URL_Checker/Trained_Model/Vectorization_Layer/vocab.json", "w") as f:
    json.dump(vocab, f)

In [ ]:
print(Batch_Size, Sequence_length, N_for_NGram, max_tokens)

# Loading the Model and Vocabulary from Saved Model

In [ ]:
modell = tf.keras.models.load_model("/content/drive/MyDrive/URL_Checker/Trained_Model/Model/Model.keras")

In [ ]:
# Batch_Size = 16 * strategy.num_replicas_in_sync
Batch_Size = 128
max_tokens = 10000
with open("/content/drive/MyDrive/URL_Checker/Trained_Model/Vectorization_Layer/vocab.json", "r") as f:
    vocab = json.load(f)
VL = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length,
                                                   vocabulary=vocab)

In [ ]:
links = ["itunes.apple.com/gb/album/whb-bonus-track-version/id358709625","http://45.231.210.144:47317/Mozi.m"]
Evaluation.make_prediction(modell,Batch_Size, VL, links, 0.5)

1/1 [==============================] - 0s 79ms/step


['Safe', 'Malicious']